In [1]:
# Account login IDs (not credentials) imported from a variables file
## Source: https://docs.microsoft.com/en-us/learn/modules/run-parallel-tasks-in-azure-batch-with-the-azure-cl

. ./AZ_Sub_Details_1.ps1

$tenantID = $TENANTID_DEF
$subscriptionID = $SUBSCRIPTIONID_DEF

# Create the Variables
$location = "EastUS2"
$resourceGroupName = "rg-npde2bat001"
$batch_Account = "batnpde2trn001"
$batch_Pool = "batchpool1"
$batch_vm_Size = "Standard_A1_v2"
$batch_vm_Sku = 
$batch_as = 3

$taskId = "mytask"
$jobId = "myjob"
$cmdLine = "/bin/bash -c 'echo \`$(printenv | grep \AZ_BATCH_TASK_ID) processed by; echo \`$(printenv | grep \AZ_BATCH_NODE_ID)'"


The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
# Login
Connect-AzAccount -Tenant $tenantID -SubscriptionId $subscriptionID


Account               SubscriptionName      TenantId                             Environment
-------               ----------------      --------                             -----------
matperr@microsoft.com MWP_Prod_Subscription 72f988bf-86f1-41af-91ab-2d7cd011db47 AzureCloud



In [ ]:
# If not registered, Register the AZResource provider for batch
## Register-AzResourceProvider -ProviderNamespace Microsoft.Batch

In [3]:
# Create the resource group and batch account
New-AzResourceGroup -Name $resourceGroupName -Location $location
New-AzBatchAccount -AccountName $batch_account -ResourceGroupName $resourceGroupName -Location $location


ResourceGroupName : rg-npde2bat001
Location          : eastus2
ProvisioningState : Succeeded
Tags              : 
ResourceId        : /subscriptions/b67872ff-3d41-4494-a27c-a8ef9c3b0d00/resourceGroups/rg-npde2bat0
                    01


Id                                    : /subscriptions/b67872ff-3d41-4494-a27c-a8ef9c3b0d00/resourc
                                        eGroups/rg-npde2bat001/providers/Microsoft.Batch/batchAccou
                                        nts/batnpde2trn001
AccountEndpoint                       : batnpde2trn001.eastus2.batch.azure.com
PrimaryAccountKey                     : 
SecondaryAccountKey                   : 
AccountName                           : batnpde2trn001
Location                              : eastus2
ResourceGroupName                     : rg-npde2bat001
Subscription                          : b67872ff-3d41-4494-a27c-a8ef9c3b0d00
State                                 : Succeeded
TaskTenantUrl                         : https://batnpde

In [4]:
# Create an Azure batch key for Key based on Authentication
##New-AzBatchAccountKey -AccountName $batch_Account -KeyType Primary # or Secondary

#Get the Azure Batch Keys
##$Account = Get-AzBatchAccountKeys –AccountName $batch_Account

##$Account.PrimaryAccountKey
##$Account.SecondaryAccountKey

# Authentication via SharedKey
$context = Get-AzBatchAccountKey -AccountName $batch_Account

# OR AAD Authentication
#$context - Get-AzBatchAcount -AccountName $batch_Account


Get-AzBatchAccountKeys alias will be removed in an upcoming breaking change release
- The change is expected to take effect from the version : '2.0.0'
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other information on breaking changes in Azure PowerShell.


In [5]:
# Create the Batch Pool
## Create a Windows Batch Pool
#$imageReference = New-Object -TypeName "Microsoft.Azure.Commands.Batch.Models.PSImageReference" -ArgumentList @("WindowsServer", "MicrosoftWindowsServer", "2016-Datacenter", "*"
#$configuration = New-Object -TypeName "Microsoft.Azure.Commands.Batch.Models.PSVirtualMachineConfiguration" -ArgumentList @($imageReference, "batch.node.windows amd64")

## Create an Ubuntu Batch Pool
$imageReference = New-Object -TypeName "Microsoft.Azure.Commands.Batch.Models.PSImageReference" -ArgumentList @("UbuntuServer","Canonical","18.04-LTS")
$configuration = New-Object -TypeName "Microsoft.Azure.Commands.Batch.Models.PSVirtualMachineConfiguration" -ArgumentList @($imageReference, "batch.node.ubuntu 18.04")

New-AzBatchPool -Id $batch_Pool -VirtualMachineSize $batch_vm_Size -TargetDedicatedComputeNodes $batch_as -VirtualMachineConfiguration $configuration -BatchContext $context

In [7]:
# Get allocation Status
$allocState = Get-AzBatchPool -Id $batch_pool -BatchContext $Context
$allocState.AllocationState

Steady


In [12]:
# Create a batch job

$PoolInformation = New-Object -TypeName "Microsoft.Azure.Commands.Batch.Models.PSPoolInformation"
$PoolInformation
$PoolInformation.PoolId = $batch_pool

New-AzBatchJob -Id $jobId -PoolInformation $PoolInformation -BatchContext $Context

for ($ctr = 1; $ctr -le 3; $ctr++)
{
   New-AzBatchTask -Id "$taskid$ctr" -JobId $jobId -CommandLine $cmdLine -BatchContext $context
}


AutoPoolSpecification PoolId
--------------------- ------
                      
New-AzBatchTask: 
Line |
  11 |     New-AzBatchTask -Id "$taskid$ctr" -JobId $jobId -CommandLine $cmdL …
     |     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | HttpStatusCode: Conflict
StatusMessage: The specified task already exists.
ClientRequestId: 
RequestId: 20311dcd-4ec1-4bd6-9039-c8d5680cede0
Error code: TaskExists
Message: The specified task already exists.
RequestId:20311dcd-4ec1-4bd6-9039-c8d5680cede0
Time:2021-02-23T16:01:51.9368288Z

New-AzBatchTask: 
Line |
  11 |     New-AzBatchTask -Id "$taskid$ctr" -JobId $jobId -CommandLine $cmdL …
     |     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | HttpStatusCode: Conflict
StatusMessage: The specified task already exists.
ClientRequestId: 
RequestId: e563c665-b333-4e47-93c7-0875828a53dd
Error code: TaskExists
Message: The specified task already exists.
RequestId:e563c665-b333-4e47-93c7-087582

In [ ]:
# Delete the Batch Job(s)
#Remove-AzBatchJob -id $jobId -BatchContext $context

In [ ]:
# Delete Azure Batch Account
##Remove-AzBatchAccount -AccountName $batch_Account